In [25]:
import pickle

import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [26]:
from sklearn.pipeline import make_pipeline

In [27]:
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("green-taxi-duration")

<Experiment: artifact_location='mlflow-artifacts:/935440909942362689', creation_time=1688121676456, experiment_id='935440909942362689', last_update_time=1688121676456, lifecycle_stage='active', name='green-taxi-duration', tags={}>

In [30]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df


def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [31]:
df_train = read_dataframe('../../data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('../../data/green_tripdata_2021-02.parquet')

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

In [32]:
with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)

    pipeline = make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params, n_jobs=-1)
    )

    pipeline.fit(dict_train, y_train)
    y_pred = pipeline.predict(dict_val)

    rmse = mean_squared_error(y_pred, y_val, squared=False)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)

    mlflow.sklearn.log_model(pipeline, artifact_path="model")

{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 6.7558229919200725


In [33]:
from mlflow.tracking import MlflowClient


In [34]:
MLFLOW_TRACKING_URI = 'http://127.0.0.1:5000'
RUN_ID = '4680bb12ecda4755ade30611daf30c48'

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [35]:
path = client.download_artifacts(run_id=RUN_ID, path='dict_vectorizer.bin')

/var/folders/6p/kwmkg6ps5d74czsbsp1p6zm80000gn/T/ipykernel_7830/2836594815.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.download_artifacts`` is deprecated since 2.0. This method will be removed in a future release. Use ``mlflow.artifacts.download_artifacts`` instead.
  path = client.download_artifacts(run_id=RUN_ID, path='dict_vectorizer.bin')


MlflowException: The following failures occurred while downloading one or more artifacts from http://127.0.0.1:5000/api/2.0/mlflow-artifacts/artifacts/935440909942362689/4680bb12ecda4755ade30611daf30c48/artifacts: {'dict_vectorizer.bin': 'MlflowException("API request to http://127.0.0.1:5000/api/2.0/mlflow-artifacts/artifacts/935440909942362689/4680bb12ecda4755ade30611daf30c48/artifacts/dict_vectorizer.bin failed with exception HTTPConnectionPool(host=\'127.0.0.1\', port=5000): Max retries exceeded with url: /api/2.0/mlflow-artifacts/artifacts/935440909942362689/4680bb12ecda4755ade30611daf30c48/artifacts/dict_vectorizer.bin (Caused by ResponseError(\'too many 500 error responses\'))")'}

In [17]:
with open(path, 'rb') as f_out:
    dv = pickle.load(f_out)

NameError: name 'path' is not defined

In [23]:
dv

DictVectorizer()